# What's on TV Tonight ?

This notebook groups every steps of the project :
- web scraping the french tv programme and get all the desired informations
    - channel number & channel name
    - programme name / title
    - beginning & duration time
    - rate /5 given by "TéléLoisirs"
    - synopsis of the programme
    - year of release
    
    
- display programmes by type (the user chooses the type of programme he wants to see), and sorted by rate for movies 


The code also works with what is on TV right now ("En ce moment" table on the main web page) and  for different TV guide (Orange, SFR, Bouygues etc.)

## Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

## URL links & Constants

In [2]:
URL_TNT = "https://www.programme-tv.net/"
URL_CANAL = "https://www.programme-tv.net/programme/canal-plus/"
URL_NOW = "https://www.programme-tv.net/programme/en-ce-moment.html"
URL_TOMORROW = "https://www.programme-tv.net/programme/toutes-les-chaines/2020-12-23/" #change everyday

CHANNEL_NAME = "channel_name"
CHANNEL_NUMBER = "channel_number"
BEGINNING_HOUR = "beginning_hour"
PROGRAMME_TITLE = "programme_title"
PROGRAMME_TYPE = "programme_type" 
PROGRAMME_DURATION = "programme_duration"
SYNOPSIS = "synopsis"
RATE = "rate /5"

## Data retrieval

###### Get data for TNT (Télévision Numérique Terrestre) 

In [3]:
def loader_tnt(url_link: str) -> pd.DataFrame:
    
    # Set up scraping tool
    url_basic = url_link
    response_basic = requests.get(url_basic)
    soup = BeautifulSoup(response_basic.text, 'html.parser')
    info_box = soup.find_all("div", class_="homeGrid-cards")
    tv_programme = []
    
    for i in range(len(info_box)):
        # Get first useful informations on the main page
        channel_name = info_box[i].find("div", class_="homeGrid-cardsChannelItem").a["title"]
        channel_number = getattr(info_box[i].find("div", class_="homeGrid-cardsChannelNumber"), 'text', None)
        beginning_hour = getattr(info_box[i].find("div", class_="mainBroadcastCard-startingHour"), 'text'
                                 , None).replace("\n", " ").strip()
        programme_title = info_box[i].find("div", class_="mainBroadcastCard-title").a["title"]
        programme_type = getattr(info_box[i].find("div", class_="mainBroadcastCard-format"), 'text'
                                 , None).replace("\n", " ").strip()
        prgrm_duration = getattr(info_box[i].find("span", class_="mainBroadcastCard-durationContent"), 'text'
                                 , None).replace("\n", " ").strip()
        url_link = info_box[i].find("div", class_="mainBroadcastCard-title").a["href"]


        # ReDo requests.get() to go through the link of the program to get new infos (synopsis / opinion) ...
        url_synopsis = url_link
        response_url_synopsis = requests.get(url_synopsis)
        soup_synopsis = BeautifulSoup(response_url_synopsis.text, 'html.parser')
        
        # ... based on different program type (movies, film, series) because the html structure is different
        movies_box = soup_synopsis.find_all("div", class_="programHome-mainContent")
        serie_sport_box = soup_synopsis.find_all("div", class_="programCollectionEpisode-mainContent")
        
        # Create 3 variables of synopsis_type : it fills when condition is met and stays blank if not 
        # Get synopsis of film OR summary (mostly for sports, culture program, series & others)
        try:
            synopsis_film = (movies_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                            ).text.strip()
        except:
            synopsis_film = " "
    
        try:
            sport_details = (serie_sport_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                            ).text.strip()
        except:
            sport_details = " "
        
        try:
            opinion = getattr(movies_box[0].find("div", class_="review-loveText"), 'text', str(None))
        except:
            opinion = "None"
        
        try:
            genre = getattr(movies_box[0].find("div", class_="overview-overviewSubtitle"), 'text', str(None))
            find_integer = str(" (") + str([int(s) for s in genre.split() if s.isdigit()][0]) + str(")")
        except:
            find_integer = " "

        info = [channel_name, programme_title, programme_type, channel_number, beginning_hour, prgrm_duration, 
                opinion, synopsis_film, sport_details, find_integer]
        
        tv_programme.append(info)

    # Build a DataFrame from scraped data
    tv_programme = pd.DataFrame(tv_programme, columns=[CHANNEL_NAME, PROGRAMME_TITLE, PROGRAMME_TYPE, 
                                                       CHANNEL_NUMBER, BEGINNING_HOUR, PROGRAMME_DURATION, 
                                                       RATE, "synopsis_film", "sport_details", "year"])
    
    # Group channel_name & channel_number together to use it as only one index column
    tv_programme["channel_name"] = (tv_programme["channel_name"] + str(" (") + 
                                    tv_programme["channel_number"] + str(")"))
    
    # Group all synopsis type (movies, sport, series) into one unique "synopsis" variable    
    tv_programme[SYNOPSIS] = (tv_programme["synopsis_film"] + tv_programme["sport_details"])
        
    # Replace str(opinion) by an integer representing the rate / 5 of the programme ...
    # couldn't scrape this variable on the site because the int is a <svg ...> (technical / knowledge issue)
    # /!\ 0 means NO RATE ; not that the porgramme is very very bad 
    tv_programme[RATE].replace({"\n                    À ne pas manquer\n                " : 5, 
                                "\n                    Très bon\n                "         : 4, 
                                "\n                    Bon\n                "              : 3, 
                                "\n                    Assez bon\n                "        : 2, 
                                "\n                    Décevant\n                "         : 1, 
                                "None": 0}, inplace=True)

    # Concatenate Title & Year of release
    tv_programme[PROGRAMME_TITLE] = tv_programme[PROGRAMME_TITLE] + tv_programme["year"]
    
    # Drop intermediate columns    
    tv_programme.drop(columns=["synopsis_film", "sport_details", "year", "channel_number"], 
                      axis=1, inplace=True)
    
    return tv_programme

In [4]:
programme_tnt = loader_tnt(URL_TNT)
# programme_tnt

###### Get data for "Le bouquet de Canal" (private channel - not necessary if you didn't subscribe) 

In [5]:
# Same function as loader_tnt with one additionnal command : lines 9-10 & 67-68
def loader_canal(url_link: str) -> pd.DataFrame:
    
    url_basic = "https://www.programme-tv.net/programme/canal-plus/"
    response_basic = requests.get(url_basic)
    soup = BeautifulSoup(response_basic.text, 'html.parser')
    # 2 find_all() because I can't reach the channel name thanks to the class_="bouquet-cards" ... I don't 
    # understand why !
    info_box_canal = soup.find_all("div", class_="mainBroadcastCard first reverse")
    info_box_channel = soup.find_all("div", class_="bouquet-cards")
    tv_programme = []
    
    for i in range(len(info_box_canal)):
        channel_name = info_box_channel[i].find("div", class_="bouquet-cardsChannelItem").a["title"]
        channel_number = ""
        beginning_hour = getattr(info_box_canal[i].find("div", class_="mainBroadcastCard-startingHour"), 
                                 'text', None).replace("\n", " ").strip()
        programme_title = info_box_canal[i].find("div", class_="mainBroadcastCard-title").a["title"]
        programme_type = getattr(info_box_canal[i].find("div", class_="mainBroadcastCard-format"), 
                                 'text', None).replace("\n", " ").strip()
        prgrm_duration = getattr(info_box_canal[i].find("span", class_="mainBroadcastCard-durationContent"), 
                                 'text', None).replace("\n", " ").strip()
        url_link = info_box_canal[i].find("div", class_="mainBroadcastCard-title").a["href"]

        url_synopsis = url_link
        response_url_synopsis = requests.get(url_synopsis)
        soup_synopsis = BeautifulSoup(response_url_synopsis.text, 'html.parser')
        
        movies_box = soup_synopsis.find_all("div", class_="programHome-mainContent")
        serie_sport_box = soup_synopsis.find_all("div", class_="programCollectionEpisode-mainContent")
        
        try:
            synopsis_film = (movies_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                            ).text.strip()
        except:
            synopsis_film = " "
    
        try:
            sport_details = (serie_sport_box[0].find("div", class_="synopsis-text defaultStyleContentTags")
                            ).text.strip()
        except:
            sport_details = " "
        
        try:
            opinion = getattr(movies_box[0].find("div", class_="review-loveText"), 'text', str(None))
        except:
            opinion = "None"
        
        try:
            genre = getattr(movies_box[0].find("div", class_="overview-overviewSubtitle"), 'text', str(None))
            find_integer = str(" (") + str([int(s) for s in genre.split() if s.isdigit()][0]) + str(")")
        except:
            find_integer = " "

        info = [channel_name, programme_title, programme_type, channel_number, beginning_hour, prgrm_duration, 
                opinion, synopsis_film, sport_details, find_integer]
        
        tv_programme.append(info)

    tv_programme = pd.DataFrame(tv_programme, columns=[CHANNEL_NAME, PROGRAMME_TITLE, PROGRAMME_TYPE, 
                                                       CHANNEL_NUMBER, BEGINNING_HOUR, PROGRAMME_DURATION, 
                                                       RATE, "synopsis_film", "sport_details", "year"])
    
    tv_programme[SYNOPSIS] = (tv_programme["synopsis_film"] + tv_programme["sport_details"])
    
    # channel_number is not given by the website for canal channels --> enter values by hand (for my version)
    tv_programme[CHANNEL_NUMBER] = ["Chaîne n°4", "Chaîne n°40", "Chaîne n°41", "Chaîne n°42", 
                                    "Chaîne n°43", "Chaîne n°44"]
    
    tv_programme["channel_name"] = (tv_programme["channel_name"] + str(" (") + 
                                    tv_programme["channel_number"] + str(")"))

    tv_programme[RATE].replace({"\n                    À ne pas manquer\n                " : 5, 
                                "\n                    Très bon\n                "         : 4, 
                                "\n                    Bon\n                "              : 3, 
                                "\n                    Assez bon\n                "        : 2, 
                                "\n                    Décevant\n                "         : 1, 
                                "None": 0}, inplace=True)

    tv_programme[PROGRAMME_TITLE] = tv_programme[PROGRAMME_TITLE] + tv_programme["year"]
    
    tv_programme.drop(columns=["synopsis_film", "sport_details", "year", "channel_number"], 
                      axis=1, inplace=True)
        
    return tv_programme

In [6]:
programme_canal = loader_canal(URL_CANAL)
# programme_canal

###### Concatenate the 2 dataframes (TNT + "Le bouquet de Canal")

In [7]:
def concatenate_frames(df_tnt: pd.DataFrame, df_canal: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([df_tnt, df_canal]).drop_duplicates(subset ="channel_name")

In [8]:
programme_tv = concatenate_frames(programme_tnt, programme_canal)
# programme_tv.head()

## Display best programmes by categorie

###### List of today's programme type ---> choose what you want to watch tonight 

In [9]:
programme_type_of_the_day = programme_tv[PROGRAMME_TYPE].unique().tolist()
programme_type_of_the_day

['Série TV', 'Culture Infos', 'Cinéma', 'Autre', 'Sport']

###### Select a dataframe (TNT, Le bouquet de Canal or both) and the programme type you want to watch

In [10]:
def choose_programme_type(df: pd.DataFrame, programme_type: str) -> pd.DataFrame:    
    pd.set_option("display.max_colwidth", -1)
    df_chosed = df.loc[df[PROGRAMME_TYPE] == programme_type]
    df_chosed = df_chosed.drop(columns=[PROGRAMME_TYPE]).set_index(CHANNEL_NAME)
    df_chosed = df_chosed.sort_values(by=RATE, ascending=False)#.head(3)
    #df_chosed = df_chosed.drop(columns=RATE)
    return df_chosed

In [11]:
selected_type = choose_programme_type(programme_tv, "Cinéma")
selected_type

,programme_title,beginning_hour,programme_duration,rate /5,synopsis
channel_name,,,,,
TF1 Séries Films (Chaîne n°20),4 mariages et un enterrement (1994),21h00,2h10min,4,"A Londres, Charles, 32 ans, est un célibataire endurci. Autour de lui, les mariages se multiplient et sa seule joie dans ces festivités est de retrouver son groupe d'amis. Parmi eux, il y a Scarlett, sa colocataire, Fiona, secrètement amoureuse de lui, Matthew et Gareth, un couple homosexuel, Tom, un aristocrate benêt, et David, son frère sourd-muet. Lors d'une noce, il fait la connaissance de Carrie, une Américaine libérée dont il s'éprend."
M6 (Chaîne n°6),Insaisissables (2013),21h05,2h05min,3,"J. Daniel Atlas, Merritt McKinney, Henley Reeves et Jack Wilder, qui vivotent de leurs talents de prestidigitateurs, reçoivent chacun une carte de tarot qui leur donne rendez-vous au même endroit. Un plan mystérieux leur est dévoilé. Un an plus tard, en direct d'une scène de Las Vegas, ils effectuent un incroyable braquage dans une banque parisienne. Alma, d'Interpol, et Dylan, du FBI, sont chargés de l'enquête."
Chérie 25 (Chaîne n°25),Contre-enquête (2006),21h05,1h35min,3,"L'inspecteur Richard Malinowski, de la brigade criminelle, est confronté chaque jour à des faits divers très sanglants. Lorsque sa fillette est violée et assassinée, son existence bascule. Les investigations aboutissent rapidement à l'arrestation d'un suspect, Daniel Eckmann. Depuis sa prison, ce dernier, qui clame son innocence, envoie une lettre à Malinowkski. Richard commence à douter de sa culpabilité."
Canal+ Décalé (Chaîne n°43),L'art du mensonge (2019),21h00,1h46min,3,"A Londres, en 2009, Estelle, veuve depuis un an, vient de s'inscrire sur un site de rencontres. C'est comme cela quelle se retrouve dans un restaurant en compagnie de Brian, septuagénaire et veuf comme elle, rencontré sur le site. Au cours du repas, Brian annonce à Estelle qu'il lui a menti : il s'appelle en fait Roy Courtnay. Estelle lui avoue alors qu'elle s'appelle Betty McLeish. Mais si Betty a caché qui elle était, et notamment la fortune dont elle est l'héritière, Roy dissimule sa véritable activité d'escroc sans scrupules."
Canal+ Family (Chaîne n°44),Stan & Ollie (2018),20h53,1h35min,3,"A l'été 1937, à Culver City, en Californie, Stan Laurel Et Oliver Hardy ne sont plus les grandes vedettes comiques qu'ils étaient à Hollywood, à l'époque où leurs films, aimés du public et des critiques, étaient doublés dans de nombreuses langues et connus à travers le monde. Les deux comédiens, toujours sous contrat avec le producteur Hal Roach, auprès duquel, Stan tente en vain d'obtenir une augmentation. Seize ans plus tard, Stan et Oliver arrivent à Newcastle, en Angleterre, pour travailler avec le producteur de spectacles Bernard Delfont."
France 3 (Chaîne n°3),C'est beau la vie quand on y pense (2017),21h05,1h31min,2,"Ancien pilote de rallye dépressif et sans travail, Loïc veut à tout prix rencontrer celui à qui a été greffé le coeur de son fils, mort dans un accident de voiture. Grâce à un ami médecin, il trouve son adresse et s'y rend. Il fait alors la connaissance d'Hugo, un jeune casse-cou à la dérive qui vit seul dans l'appartement de sa grand-mère. Voulant fuir la police après un braquage raté, Hugo lui rend ensuite visite, en Bretagne et fait un malaise devant la porte. Loïc lui propose alors de rester. Ensemble, ils essayent de réparer l'ancienne voiture de course de Loïc."
TMC (Chaîne n°10),Taxi 5 (2018),21h15,1h55min,2,"Sylvain Marot, policier parisien intrépide doublé d'un pilote émérite, a une façon bien à lui de faire parler les suspects. Mais son comportement hors du service lui cause des soucis : pour avoir approché de trop près la femme du préfet, Marot est immédiatement muté à Marseille. Le policier découvre avec effroi un commissariat qui ne répond pas à ses normes d'efficacité. Lors de sa première sortie en ville, il fait rapidement connaissance d'Eddy Maklouf, chauffeur de taxi qui se moque du code de la route

### Few ints for the webapp

[Advanced callbacks](https://dash.plotly.com/advanced-callbacks)

[Return a dataframe as a data_table from a callback](https://stackoverflow.com/questions/55269763/return-a-pandas-dataframe-as-a-data-table-from-a-callback-with-plotly-dash-for-p/55305812#55305812)

[Return df with dash](https://dash.plotly.com/datatable/editable)

In [12]:
choices = {
    "TONIGHT": {"TNT": "https://www.programme-tv.net/", "CANAL": "https://www.programme-tv.net/programme/canal-plus/"},
    "NOW"    : {"TNT":"https://www.programme-tv.net/programme/en-ce-moment.html", "CANAL": "https://www.programme-tv.net/programme/canal-plus/en-ce-moment.html"}
}

In [13]:
choices.keys()

dict_keys(['TONIGHT', 'NOW'])

In [14]:
choices["TONIGHT"]

{'TNT': 'https://www.programme-tv.net/',
 'CANAL': 'https://www.programme-tv.net/programme/canal-plus/'}

In [15]:
choices["TONIGHT"]["TNT"]

'https://www.programme-tv.net/'

In [16]:
choices["TONIGHT"]["CANAL"]

'https://www.programme-tv.net/programme/canal-plus/'